# IBM Data Science Professional Certificate
## Capstone project Part 1-'Segmenting - Clustering Neighborhoods in Toronto'
### By Zahra Adahman

In [4]:
# @hidden_cell
USERNAME='Vivazee'
PASSWORD='.FVgebd93QvdvN3R!#'
print ('Hello Capstone Project Course!')


Hello Capstone Project Course!


# Question 1- Scrape the data from wiki page and generate dataframe

## Step 1

In [5]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [6]:
# List of Postal codes in Canada data from Wikipedia
CA_PC_wiki='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
UnPro_CA_PC_wiki= requests.get(CA_PC_wiki).text

# beautiful soup to pull data from the HTML/XML codes.
soupCA_PC = BeautifulSoup(UnPro_CA_PC_wiki,'xml')
#print(soupCA_PC)

## Step 2
### a. Processing data in table

In [7]:
# Creating a clean df table from soupCA_PC
data = []
columns = []
CA_PCtable = soupCA_PC.find(class_='wikitable')
for index, tr in enumerate(CA_PCtable.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row is header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list to df
CA_PCdf = pd.DataFrame(data = data,columns = columns)
CA_PCdf.head()
#CA_PCdf.shape
#CA_PCdf.dtypes

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [8]:
#Remove rows with boroughs 'Not assigned'
CA_PCdf1 =CA_PCdf[CA_PCdf['Borough'] != 'Not assigned']
CA_PCdf1.head()
#CA_PCdf1.shape


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### b. Removing duplicates neighbourhoods.

In [9]:
#Groupby: neighborhood by postal code and drop duplicates
CA_PCdf1["Neighborhood"] = CA_PCdf1.groupby("Postal code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))
CA_PCdf1 = CA_PCdf1.drop_duplicates()

#Change index to be postal code
if(CA_PCdf1.index.name != 'Postal code'):
    CA_PCdf1 = CA_PCdf1.set_index('Postal code')
    
CA_PCdf1.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Borough,Neighborhood
Postal code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [11]:
CA_PCdf2 =CA_PCdf[CA_PCdf['Borough'] != 'Not assigned']
CA_PCdf2["Neighborhood"] = CA_PCdf2.groupby("Postal code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))
CA_PCdf2 = CA_PCdf2.drop_duplicates()
CA_PCdf2.head()



/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [12]:
CA_PCdf2.shape

(103, 3)

# Question 2 - Generate new dataframe with coordinates

In [13]:
pip install geopy



The following command must be run outside of the IPython shell:

    $ pip install geopy

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [14]:
from  geopy.geocoders import Nominatim
CAgeolocator = Nominatim()
location = CAgeolocator.geocode("Toronto, North York, Parkwoods")

print(location.address,'latitude and longitude are:', (location.latitude, location.longitude))


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 2X2, Canada latitude and longitude are: (43.7587999, -79.3201966)


In [15]:
CA_PCgeodf = CA_PCdf2
CA_PCgeodf.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


## Decided to just use pandas to merge the geo file with the coordinates

In [22]:
CA_PCdf2.rename(columns={'Postal code': 'Postal Code'}, inplace=True)
CA_PCdf2.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [26]:
#using just pandas here to import geospatial coordinates file

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_33f2a4c0002b426c919d118a84db288b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='9waIBTtcCFTX72OMH_zEFGXObvxBkAiZJH0yQtldKMNs',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_33f2a4c0002b426c919d118a84db288b.get_object(Bucket='ibmdscoursemodule8-donotdelete-pr-chtbwxhr8kt9bj',Key='Geospatial_CoordinatesCA.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
CA_PCLLdf = pd.read_csv(body)
CA_PCLLdf.head()
#CA_PCLLdf.shape


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [34]:
#merge the geospatial file with the dataframe from Wiki
CA_PCLLdf2 = pd.merge(CA_PCdf2, CA_PCLLdf, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)
CA_PCLLdf2.head()
#CA_PCLLdf2

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


 # Question 3: Visualize, explore and cluster the neighborhoods in Toronto.

In [36]:
import json 
import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans


In [45]:
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [51]:
address = 'Toronto, CA'

CAgeolocator = Nominatim(user_agent="ca_explorer")
location = CAgeolocator.geocode(address)
latitude1 = location.latitude
longitude1 = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude1, longitude1))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# Let's explore Toronto

In [47]:
#Map Toronto from CA_PCLLdf2
map_toronto = folium.Map(location=[latitude1, longitude1], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(CA_PCLLdf2['Latitude'], CA_PCLLdf2['Longitude'], CA_PCLLdf2['Borough'], CA_PCLLdf2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [48]:

# @hidden_cell
CLIENT_ID = 'TG1UABJVFI4QA2CAICLRIPYEMN2J0QR0OZCUXRRET2PZDQZO'
CLIENT_SECRET = 'ESJFP4MYMYOWSFGSCXEOBDUSVP5MQWLBMJTHS2GZSACF5WR5'
VERSION = '20180604' # Foursquare API version

## Let's look into the North York borough

In [50]:
#Create North York dataframe
NorthYork_df = CA_PCLLdf2[CA_PCLLdf2['Borough'] == 'North York'].reset_index(drop=True)
NorthYork_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,Fairview / Henry Farm / Oriole,43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,York Mills / Silver Hills,43.757490,-79.374714
4,M2M,North York,Willowdale / Newtonbrook,43.789053,-79.408493


In [52]:
address = 'North York, Toronto'

CAgeolocator = Nominatim(user_agent="ca_explorer")
location1 = CAgeolocator.geocode(address)
latitude2 = location1.latitude
longitude2 = location1.longitude
print('The geograpical coordinate of North York, Toronto are {}, {}.'.format(latitude2, longitude2))

The geograpical coordinate of North York, Toronto are 43.7543263, -79.44911696639593.


In [55]:
#Map North York, Toronto' from NorthYork_df
map_NorthYorkTO = folium.Map(location1=[latitude2, longitude2], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(NorthYork_df['Latitude'], NorthYork_df['Longitude'], NorthYork_df['Borough'], NorthYork_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NorthYorkTO)
    
map_NorthYorkTO